# Chaining Experiments

Setting up langchain imports.

In [1]:
from langchain_openai import OpenAI;
from langchain.prompts import PromptTemplate;
from langchain_core.output_parsers import StrOutputParser;
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnablePick ;
from langchain.chains.sequential import SequentialChain;
from operator import itemgetter
from dotenv import load_dotenv;
load_dotenv();

Using OpenAI

In [2]:
llm = OpenAI();

Code Chain
```mermaid
graph TD;
    language & task-->code_chain;
    code_chain-->code;
```

In [3]:
code_prompt = PromptTemplate.from_template("Write a very short {language} function that will {task}");
code_chain = code_prompt | llm | {"code": StrOutputParser()};

print(code_chain.input_schema.schema());
print(code_chain.output_schema.schema());

{'title': 'PromptInput', 'type': 'object', 'properties': {'language': {'title': 'Language', 'type': 'string'}, 'task': {'title': 'Task', 'type': 'string'}}}
{'title': 'RunnableParallel<code>Output', 'type': 'object', 'properties': {'code': {'title': 'Code', 'type': 'string'}}}


Test Chain

```mermaid
graph TD;
    language & code-->test_chain;
    test_chain-->test
```

In [4]:
test_prompt = PromptTemplate.from_template("Write a test for the following {language} code:\n{code}");
test_chain = test_prompt | llm | {"test": StrOutputParser()};

print(test_chain.input_schema.schema());
print(test_chain.output_schema.schema());

{'title': 'PromptInput', 'type': 'object', 'properties': {'code': {'title': 'Code', 'type': 'string'}, 'language': {'title': 'Language', 'type': 'string'}}}
{'title': 'RunnableParallel<test>Output', 'type': 'object', 'properties': {'test': {'title': 'Test', 'type': 'string'}}}


Chaining Code and Test together

```mermaid
graph TD;
    language & task-->code_chain;
    code_chain-->code;
    language & code-->test_chain;
    test_chain-->test
```

In [5]:
chain = code_chain | test_chain;

print(chain.input_schema.schema())
print(chain.output_schema.schema())

{'title': 'PromptInput', 'type': 'object', 'properties': {'language': {'title': 'Language', 'type': 'string'}, 'task': {'title': 'Task', 'type': 'string'}}}
{'title': 'RunnableParallel<test>Output', 'type': 'object', 'properties': {'test': {'title': 'Test', 'type': 'string'}}}


Executing the chained chains does not work because the `language` input of the `code_chain` is not passed through to the `test_chain`. [How can it be passed through?](https://stackoverflow.com/questions/78503869/how-to-pass-input-parameters-through-runnablesequences-in-langchain-v0-2)

In [6]:
result = chain.invoke({
    "language": "python",
    "task": "reverse a string",
});

print(result);

KeyError: "Input to PromptTemplate is missing variables {'language'}.  Expected: ['code', 'language'] Received: ['code']"